# 31-JSON与XML处理

## 📚 用途说明

**学习目标**：
- 掌握Python JSON数据的序列化和反序列化
- 熟练使用xml.etree.ElementTree处理XML数据
- 理解JSON schema验证和数据结构转换
- 能够处理配置文件、API响应等实际应用场景

**前置要求**：
- 已完成30-正则表达式应用学习
- 熟练掌握Python字典、列表等数据结构
- 了解Python标准库的基本使用方法

**与LangChain关联**：
- JSON处理是LangChain结构化输出的基础
- XML处理对于LangChain的文档解析和集成至关重要
- 数据序列化影响LangChain的缓存和存储机制
- 为后续学习LangChain的API数据处理做准备

---

## 🔢 知识点覆盖

### 4.4 JSON与XML处理 [⭐基础]
**知识点说明**：JSON和XML是常用的数据交换格式，掌握它们的处理方法对于配置文件管理、API数据处理、数据持久化等实际应用场景非常重要。Python提供了强大的标准库来处理这些格式。

**学习要求**：
- 掌握json模块的基本用法和数据转换
- 熟练使用xml.etree.ElementTree处理XML数据
- 理解JSON schema验证和错误处理
- 能够处理复杂的数据结构和嵌套格式

**案例要求**：
- 实现完整的JSON和XML处理示例
- 进行数据序列化和反序列化练习
- 应用数据处理解决实际问题
- 验证点：能独立处理JSON/XML数据转换任务

In [ ]:
print("📋 JSON基础操作:")
print("=" * 50)

import json
import xml.etree.ElementTree as ET
from xml.dom import minidom
from typing import Dict, List, Any, Optional, Union
from dataclasses import dataclass, asdict
from datetime import datetime
import re

# 1. JSON基本操作
print(f"📝 1. JSON基本操作:")

# 1.1 JSON序列化 (Python对象 -> JSON字符串)
print(f"\n   🔄 1.1 JSON序列化:")

# 基本数据类型
python_data = {
    "name": "张三",
    "age": 25,
    "is_student": True,
    "courses": ["Python", "JavaScript", "SQL"],
    "address": {
        "city": "北京",
        "district": "朝阳区"
    },
    "scores": None
}

# 序列化为JSON字符串
json_str = json.dumps(python_data, ensure_ascii=False, indent=2)
print(f"   Python对象: {python_data}")
print(f"   JSON字符串:\n{json_str}")

# 1.2 JSON反序列化 (JSON字符串 -> Python对象)
print(f"\n   🔍 1.2 JSON反序列化:")

# 从JSON字符串解析
parsed_data = json.loads(json_str)
print(f"   JSON字符串: {json_str}")
print(f"   解析后的Python对象: {parsed_data}")
print(f"   数据类型: {type(parsed_data)}")
print(f"   姓名类型: {type(parsed_data['name'])}")
print(f"   年龄类型: {type(parsed_data['age'])}")
print(f"   学生类型: {type(parsed_data['is_student'])}")

# 1.3 文件操作
print(f"\n   📁 1.3 JSON文件操作:")

# 写入JSON文件
with open('student_data.json', 'w', encoding='utf-8') as f:
    json.dump(python_data, f, ensure_ascii=False, indent=2)
print(f"   JSON数据已写入 student_data.json")

# 从JSON文件读取
with open('student_data.json', 'r', encoding='utf-8') as f:
    loaded_data = json.load(f)
print(f"   从文件读取的数据: {loaded_data}")

# 1.4 自定义JSON编码器
print(f"\n   🔧 1.4 自定义JSON编码器:")

class DateTimeEncoder(json.JSONEncoder):
    """自定义JSON编码器，处理datetime对象"""
    
    def default(self, obj):
        if isinstance(obj, datetime):
            return obj.isoformat()
        elif isinstance(obj, set):
            return list(obj)
        return super().default(obj)

# 包含特殊对象的Python数据
complex_data = {
    "user_id": 12345,
    "username": "test_user",
    "created_at": datetime.now(),
    "tags": {"python", "json", "encoding"},
    "metadata": {
        "version": "1.0",
        "last_updated": datetime(2024, 12, 25, 10, 30, 0)
    }
}

# 使用自定义编码器
try:
    json_str_complex = json.dumps(complex_data, cls=DateTimeEncoder, ensure_ascii=False, indent=2)
    print(f"   复杂数据JSON序列化成功")
    print(f"   序列化结果:\n{json_str_complex}")
except Exception as e:
    print(f"   序列化失败: {e}")

# 1.5 JSON数据验证
print(f"\n   ✅ 1.5 JSON数据验证:")

def validate_json_structure(data: Dict, schema: Dict) -> bool:
    """简单的JSON结构验证"""
    for key, expected_type in schema.items():
        if key not in data:
            print(f"   缺少必需字段: {key}")
            return False
        
        if expected_type == 'string' and not isinstance(data[key], str):
            print(f"   字段 {key} 应为字符串类型")
            return False
        elif expected_type == 'number' and not isinstance(data[key], (int, float)):
            print(f"   字段 {key} 应为数字类型")
            return False
        elif expected_type == 'boolean' and not isinstance(data[key], bool):
            print(f"   字段 {key} 应为布尔类型")
            return False
        elif expected_type == 'array' and not isinstance(data[key], list):
            print(f"   字段 {key} 应为数组类型")
            return False
        elif expected_type == 'object' and not isinstance(data[key], dict):
            print(f"   字段 {key} 应为对象类型")
            return False
    
    return True

# 定义数据模式
user_schema = {
    "name": "string",
    "age": "number",
    "is_student": "boolean",
    "courses": "array",
    "address": "object"
}

# 验证数据
print(f"   验证用户数据结构:")
is_valid = validate_json_structure(parsed_data, user_schema)
print(f"   验证结果: {'通过' if is_valid else '失败'}")

# 测试无效数据
invalid_data = {
    "name": "李四",
    "age": "25",  # 错误：字符串而非数字
    "is_student": "true",  # 错误：字符串而非布尔值
    "courses": "Python",  # 错误：字符串而非数组
    "address": "北京市"  # 错误：字符串而非对象
}

print(f"\n   验证无效数据结构:")
is_valid = validate_json_structure(invalid_data, user_schema)
print(f"   验证结果: {'通过' if is_valid else '失败'}")

# 2. JSON高级操作
print(f"\n📝 2. JSON高级操作:")

# 2.1 嵌套数据提取
print(f"\n   🔍 2.1 嵌套数据提取:")

def extract_nested_value(data: Dict, path: str, default=None):
    """从嵌套字典中提取值"""
    keys = path.split('.')
    current = data
    
    for key in keys:
        if isinstance(current, dict) and key in current:
            current = current[key]
        elif isinstance(current, list) and key.isdigit():
            index = int(key)
            if 0 <= index < len(current):
                current = current[index]
            else:
                return default
        else:
            return default
    
    return current

# 复杂嵌套数据
nested_data = {
    "company": {
        "name": "科技公司",
        "employees": [
            {
                "id": 1,
                "name": "张三",
                "department": {
                    "name": "研发部",
                    "budget": 100000
                }
            },
            {
                "id": 2,
                "name": "李四",
                "department": {
                    "name": "市场部",
                    "budget": 80000
                }
            }
        ]
    }
}

# 提取嵌套值
extracted_values = {
    "公司名称": extract_nested_value(nested_data, "company.name"),
    "第一个员工姓名": extract_nested_value(nested_data, "company.employees.0.name"),
    "第二个部门预算": extract_nested_value(nested_data, "company.employees.1.department.budget"),
    "不存在的路径": extract_nested_value(nested_data, "company.manager.name", "默认值")
}

print(f"   嵌套数据: {json.dumps(nested_data, ensure_ascii=False, indent=2)}")
print(f"\n   提取结果:")
for key, value in extracted_values.items():
    print(f"     {key}: {value}")

# 2.2 JSON数据转换
print(f"\n   🔄 2.2 JSON数据转换:")

def flatten_json(data: Dict, parent_key: str = '', sep: str = '.') -> Dict:
    """将嵌套JSON展平"""
    items = []
    
    for key, value in data.items():
        new_key = f"{parent_key}{sep}{key}" if parent_key else key
        
        if isinstance(value, dict):
            items.extend(flatten_json(value, new_key, sep).items())
        elif isinstance(value, list):
            for i, item in enumerate(value):
                if isinstance(item, dict):
                    items.extend(flatten_json(item, f"{new_key}{sep}{i}", sep).items())
                else:
                    items.append((f"{new_key}{sep}{i}", item))
        else:
            items.append((new_key, value))
    
    return dict(items)

# 展平JSON数据
flattened_data = flatten_json(nested_data)
print(f"   展平后的数据:")
for key, value in flattened_data.items():
    print(f"     {key}: {value}")

# 2.3 JSON数据过滤
print(f"\n   🔍 2.3 JSON数据过滤:")

def filter_json(data: Union[Dict, List], filter_func) -> Union[Dict, List]:
    """根据条件过滤JSON数据"""
    if isinstance(data, dict):
        return {k: filter_json(v, filter_func) for k, v in data.items() if filter_func(k, v)}
    elif isinstance(data, list):
        return [filter_json(item, filter_func) for item in data if filter_func(None, item)]
    else:
        return data

# 过滤函数示例
def filter_non_empty(key, value):
    """过滤非空值"""
    if value is None or value == "":
        return False
    if isinstance(value, (dict, list)) and len(value) == 0:
        return False
    return True

def filter_numeric_values(key, value):
    """只保留数值类型"""
    return isinstance(value, (int, float))

# 测试数据
test_data = {
    "id": 1,
    "name": "测试",
    "score": 95.5,
    "empty_field": "",
    "null_field": None,
    "empty_list": [],
    "metadata": {
        "created": "2024-12-25",
        "updated": None
    }
}

print(f"   原始数据: {json.dumps(test_data, ensure_ascii=False, indent=2)}")

filtered_data = filter_json(test_data, filter_non_empty)
print(f"\n   过滤空值后: {json.dumps(filtered_data, ensure_ascii=False, indent=2)}")

# 3. 实际应用示例
print(f"\n📝 3. 实际应用示例:")

# 3.1 配置文件管理
print(f"\n   ⚙️ 3.1 配置文件管理:")

class ConfigManager:
    """配置文件管理器"""
    
    def __init__(self, config_file: str = 'config.json'):
        self.config_file = config_file
        self.config = self.load_config()
    
    def load_config(self) -> Dict:
        """加载配置文件"""
        try:
            with open(self.config_file, 'r', encoding='utf-8') as f:
                return json.load(f)
        except FileNotFoundError:
            # 创建默认配置
            default_config = {
                "database": {
                    "host": "localhost",
                    "port": 5432,
                    "name": "myapp",
                    "user": "admin",
                    "password": "password"
                },
                "api": {
                    "base_url": "https://api.example.com",
                    "timeout": 30,
                    "retry_count": 3
                },
                "logging": {
                    "level": "INFO",
                    "file": "app.log",
                    "max_size": "10MB"
                }
            }
            self.save_config(default_config)
            return default_config
    
    def save_config(self, config: Dict = None):
        """保存配置文件"""
        if config is None:
            config = self.config
        
        with open(self.config_file, 'w', encoding='utf-8') as f:
            json.dump(config, f, ensure_ascii=False, indent=2)
    
    def get(self, path: str, default=None):
        """获取配置值"""
        return extract_nested_value(self.config, path, default)
    
    def set(self, path: str, value):
        """设置配置值"""
        keys = path.split('.')
        current = self.config
        
        for key in keys[:-1]:
            if key not in current:
                current[key] = {}
            current = current[key]
        
        current[keys[-1]] = value
        self.save_config()
    
    def update(self, updates: Dict):
        """批量更新配置"""
        def deep_update(base_dict, update_dict):
            for key, value in update_dict.items():
                if isinstance(value, dict) and key in base_dict and isinstance(base_dict[key], dict):
                    deep_update(base_dict[key], value)
                else:
                    base_dict[key] = value
        
        deep_update(self.config, updates)
        self.save_config()

# 测试配置管理器
config_manager = ConfigManager()

print(f"   配置文件已创建/加载")
print(f"   数据库主机: {config_manager.get('database.host')}")
print(f"   API超时: {config_manager.get('api.timeout')}")
print(f"   日志级别: {config_manager.get('logging.level')}")

# 更新配置
config_manager.set('database.port', 3306)
config_manager.set('api.timeout', 60)

print(f"\n   更新后配置:")
print(f"   数据库端口: {config_manager.get('database.port')}")
print(f"   API超时: {config_manager.get('api.timeout')}")

# 批量更新
config_manager.update({
    "database": {
        "user": "new_user",
        "password": "new_password"
    },
    "new_section": {
        "feature_enabled": True,
        "max_connections": 100
    }
})

print(f"\n   批量更新后:")
print(f"   数据库用户: {config_manager.get('database.user')}")
print(f"   新功能启用: {config_manager.get('new_section.feature_enabled')}")

print(f"\n✅ JSON基础操作完成")
print(f"🎯 学习目标达成:")
print(f"   ✓ 掌握json模块的基本用法和数据转换")
print(f"   ✓ 理解JSON schema验证和错误处理")
print(f"   ✓ 能处理复杂的数据结构和嵌套格式")
print(f"   ✓ 掌握实际应用场景中的配置文件管理")

### XML处理基础 [⭐基础]
**知识点说明**：XML是另一种重要的数据交换格式，特别是在配置文件、文档处理、Web服务等领域。Python的xml.etree.ElementTree模块提供了简洁而强大的XML处理能力。

**学习要求**：
- 掌握ElementTree的基本操作和方法
- 熟练进行XML解析和数据提取
- 理解XML的层级结构和命名空间
- 能够创建和修改XML文档

**案例要求**：
- 实现完整的XML处理示例
- 进行XML解析和数据提取练习
- 应用XML处理解决实际问题
- 验证点：能独立处理XML数据解析任务

In [ ]:
print("📋 XML基础操作:")
print("=" * 50)

import xml.etree.ElementTree as ET
from xml.dom import minidom
from typing import Dict, List, Any, Optional, Union
import re

# 1. XML基本操作
print(f"📝 1. XML基本操作:")

# 1.1 创建XML文档
print(f"\n   🏗️ 1.1 创建XML文档:")

# 创建根元素
root = ET.Element("bookstore")

# 添加书籍
book1 = ET.SubElement(root, "book", {"category": "programming"})
ET.SubElement(book1, "title").text = "Python编程"
ET.SubElement(book1, "author").text = "张三"
ET.SubElement(book1, "year").text = "2024"
ET.SubElement(book1, "price").text = "89.90"

book2 = ET.SubElement(root, "book", {"category": "data_science"})
ET.SubElement(book2, "title").text = "数据分析实战"
ET.SubElement(book2, "author").text = "李四"
ET.SubElement(book2, "year").text = "2023"
ET.SubElement(book2, "price").text = "79.90"

# 生成XML字符串
xml_str = ET.tostring(root, encoding='unicode')
print(f"   生成的XML:\n{xml_str}")

# 美化XML格式
def prettify_xml(element):
    """美化XML格式"""
    rough_string = ET.tostring(element, encoding='unicode')
    reparsed = minidom.parseString(rough_string)
    return reparsed.toprettyxml(indent="  ")

pretty_xml = prettify_xml(root)
print(f"\n   美化后的XML:\n{pretty_xml}")

# 保存XML文件
tree = ET.ElementTree(root)
tree.write('bookstore.xml', encoding='utf-8', xml_declaration=True)
print(f"\n   XML已保存到 bookstore.xml")

# 1.2 解析XML文档
print(f"\n   🔍 1.2 解析XML文档:")

# 从文件解析XML
tree = ET.parse('bookstore.xml')
parsed_root = tree.getroot()

print(f"   根元素标签: {parsed_root.tag}")
print(f"   根元素属性: {parsed_root.attrib}")
print(f"   子元素数量: {len(parsed_root)}")

# 遍历所有书籍
print(f"\n   所有书籍信息:")
for book in parsed_root:
    category = book.get('category', '未知')
    title = book.find('title').text
    author = book.find('author').text
    year = book.find('year').text
    price = book.find('price').text
    print(f"     书名: {title}, 作者: {author}, 年份: {year}, 价格: {price}, 类别: {category}")

# 1.3 XML数据提取
print(f"\n   📊 1.3 XML数据提取:")

# 使用XPath查找元素
print(f"\n   XPath查询示例:")

# 查找所有书籍
all_books = parsed_root.findall('book')
print(f"   所有书籍数量: {len(all_books)}")

# 查找编程类书籍
programming_books = parsed_root.findall("book[@category='programming']")
print(f"   编程类书籍数量: {len(programming_books)}")
for book in programming_books:
    title = book.find('title').text
    print(f"     编程类书籍: {title}")

# 查找所有标题
titles = parsed_root.findall('.//title')
print(f"\n   所有标题:")
for title in titles:
    print(f"     {title.text}")

# 查找价格大于80的书籍
expensive_books = []
for book in parsed_root.findall('book'):
    price_elem = book.find('price')
    if price_elem is not None and float(price_elem.text) > 80:
        expensive_books.append({
            'title': book.find('title').text,
            'price': price_elem.text
        })

print(f"\n   价格大于80的书籍:")
for book in expensive_books:
    print(f"     {book['title']}: {book['price']}")

# 1.4 XML数据修改
print(f"\n   ✏️ 1.4 XML数据修改:")

# 修改书籍价格
for book in parsed_root.findall('book'):
    price_elem = book.find('price')
    if price_elem is not None:
        current_price = float(price_elem.text)
        new_price = current_price * 0.9  # 打9折
        price_elem.text = f"{new_price:.2f}"

# 添加新属性
for book in parsed_root.findall('book'):
    book.set('discounted', 'true')

# 添加新书籍
new_book = ET.SubElement(parsed_root, "book", {"category": "web_development"})
ET.SubElement(new_book, "title").text = "Web开发实战"
ET.SubElement(new_book, "author").text = "王五"
ET.SubElement(new_book, "year").text = "2024"
ET.SubElement(new_book, "price").text = "69.90"

# 保存修改后的XML
tree.write('bookstore_updated.xml', encoding='utf-8', xml_declaration=True)
print(f"   修改后的XML已保存到 bookstore_updated.xml")

# 显示修改后的结果
print(f"\n   修改后的书籍信息:")
for book in parsed_root:
    category = book.get('category', '未知')
    title = book.find('title').text
    price = book.find('price').text
    discounted = book.get('discounted', 'false')
    print(f"     {title} - {price} ({'已打折' if discounted == 'true' else '原价'}) - {category}")

# 2. XML高级操作
print(f"\n📝 2. XML高级操作:")

# 2.1 XML与Python对象转换
print(f"\n   🔄 2.1 XML与Python对象转换:")

def xml_to_dict(element):
    """将XML元素转换为字典"""
    result = {}
    
    # 添加属性
    if element.attrib:
        result.update(element.attrib)
    
    # 处理子元素
    children = list(element)
    if children:
        child_dict = {}
        for child in children:
            child_data = xml_to_dict(child)
            
            if child.tag in child_dict:
                # 如果已存在相同标签，转换为列表
                if not isinstance(child_dict[child.tag], list):
                    child_dict[child.tag] = [child_dict[child.tag]]
                child_dict[child.tag].append(child_data)
            else:
                child_dict[child.tag] = child_data
        
        result.update(child_dict)
    
    # 添加文本内容
    if element.text and element.text.strip():
        if children or element.attrib:
            result['text'] = element.text.strip()
        else:
            return element.text.strip()
    
    return result

def dict_to_xml(data, root_name='root'):
    """将字典转换为XML元素"""
    if isinstance(data, dict):
        element = ET.Element(root_name)
        
        for key, value in data.items():
            if key == 'text':
                element.text = str(value)
            elif isinstance(value, dict):
                child = dict_to_xml(value, key)
                element.append(child)
            elif isinstance(value, list):
                for item in value:
                    child = dict_to_xml(item, key)
                    element.append(child)
            else:
                child = ET.SubElement(element, key)
                child.text = str(value)
        
        return element
    else:
        element = ET.Element(root_name)
        element.text = str(data)
        return element

# 测试XML转字典
xml_dict = xml_to_dict(parsed_root)
print(f"   XML转字典结果:")
print(f"   {json.dumps(xml_dict, ensure_ascii=False, indent=2)}")

# 测试字典转XML
test_dict = {
    "student": {
        "name": "张三",
        "age": 20,
        "courses": {
            "course": [
                {"name": "Python", "score": 90},
                {"name": "Math", "score": 85}
            ]
        }
    }
}

xml_from_dict = dict_to_xml(test_dict, 'students')
pretty_dict_xml = prettify_xml(xml_from_dict)
print(f"\n   字典转XML结果:\n{pretty_dict_xml}")

# 2.2 XML命名空间处理
print(f"\n   🏷️ 2.2 XML命名空间处理:")

# 创建带命名空间的XML
ns_xml = '''<?xml version="1.0" encoding="UTF-8"?>
<root xmlns:book="http://example.com/book" 
      xmlns:author="http://example.com/author">
    <book:book category="programming">
        <book:title>Python编程</book:title>
        <author:name>张三</author:name>
    </book:book>
</root>'''

# 解析带命名空间的XML
ns_root = ET.fromstring(ns_xml)

# 定义命名空间
namespaces = {
    'book': 'http://example.com/book',
    'author': 'http://example.com/author'
}

print(f"   带命名空间的XML解析:")
print(f"   根元素: {ns_root.tag}")

# 使用命名空间查找元素
book_elem = ns_root.find('book:book', namespaces)
if book_elem is not None:
    title_elem = book_elem.find('book:title', namespaces)
    author_elem = book_elem.find('author:name', namespaces)
    
    print(f"   书名: {title_elem.text if title_elem is not None else 'None'}")
    print(f"   作者: {author_elem.text if author_elem is not None else 'None'}")

# 2.3 XML数据验证
print(f"\n   ✅ 2.3 XML数据验证:")

def validate_xml_structure(element, required_elements):
    """验证XML结构"""
    errors = []
    
    for elem_path, elem_type in required_elements.items():
        parts = elem_path.split('/')
        current = element
        
        for part in parts:
            found = current.find(part) if current is not None else None
            if found is None:
                errors.append(f"缺少必需元素: {elem_path}")
                break
            current = found
        
        if current is not None:
            # 验证元素类型
            if elem_type == 'number':
                try:
                    float(current.text)
                except (ValueError, TypeError):
                    errors.append(f"元素 {elem_path} 应为数字类型")
            elif elem_type == 'integer':
                try:
                    int(current.text)
                except (ValueError, TypeError):
                    errors.append(f"元素 {elem_path} 应为整数类型")
            elif elem_type == 'non_empty':
                if not current.text or not current.text.strip():
                    errors.append(f"元素 {elem_path} 不能为空")
    
    return errors

# 定义书籍XML的必需结构
book_schema = {
    'book/title': 'non_empty',
    'book/author': 'non_empty',
    'book/year': 'integer',
    'book/price': 'number'
}

# 验证第一个书籍元素
first_book = parsed_root.find('book')
if first_book is not None:
    validation_errors = validate_xml_structure(first_book, book_schema)
    
    print(f"   XML结构验证结果:")
    if validation_errors:
        for error in validation_errors:
            print(f"     ❌ {error}")
    else:
        print(f"     ✅ 验证通过")

# 3. 实际应用示例
print(f"\n📝 3. 实际应用示例:")

# 3.1 RSS/Atom feed解析
print(f"\n   📰 3.1 RSS feed解析:")

class RSSParser:
    """RSS解析器"""
    
    def __init__(self):
        self.namespaces = {
            'rdf': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#',
            'dc': 'http://purl.org/dc/elements/1.1/',
            'content': 'http://purl.org/rss/1.0/modules/content/'
        }
    
    def parse_rss(self, xml_content: str) -> Dict:
        """解析RSS内容"""
        try:
            root = ET.fromstring(xml_content)
            
            # 获取频道信息
            channel = root.find('channel')
            if channel is None:
                return {'error': 'Invalid RSS format'}
            
            feed_info = {
                'title': self.get_text(channel, 'title'),
                'description': self.get_text(channel, 'description'),
                'link': self.get_text(channel, 'link'),
                'language': self.get_text(channel, 'language'),
                'last_build_date': self.get_text(channel, 'lastBuildDate'),
                'items': []
            }
            
            # 获取文章列表
            items = channel.findall('item')
            for item in items:
                item_info = {
                    'title': self.get_text(item, 'title'),
                    'description': self.get_text(item, 'description'),
                    'link': self.get_text(item, 'link'),
                    'pub_date': self.get_text(item, 'pubDate'),
                    'guid': self.get_text(item, 'guid'),
                    'categories': [cat.text for cat in item.findall('category') if cat.text]
                }
                feed_info['items'].append(item_info)
            
            return feed_info
            
        except ET.ParseError as e:
            return {'error': f'Parse error: {e}'}
        except Exception as e:
            return {'error': f'Unexpected error: {e}'}
    
    def get_text(self, element, tag):
        """安全获取元素文本"""
        found = element.find(tag)
        return found.text if found is not None and found.text else ''
    
    def extract_articles_by_category(self, feed_data: Dict, category: str) -> List[Dict]:
        """按分类提取文章"""
        filtered_items = []
        
        for item in feed_data.get('items', []):
            if category.lower() in [cat.lower() for cat in item.get('categories', [])]:
                filtered_items.append(item)
        
        return filtered_items

# 示例RSS内容
sample_rss = '''<?xml version="1.0" encoding="UTF-8"?>
<rss version="2.0">
    <channel>
        <title>技术博客</title>
        <description>最新技术文章</description>
        <link>https://techblog.example.com</link>
        <language>zh-cn</language>
        <lastBuildDate>Tue, 25 Dec 2024 10:30:00 GMT</lastBuildDate>
        
        <item>
            <title>Python异步编程详解</title>
            <description>深入理解Python的asyncio库和异步编程模式</description>
            <link>https://techblog.example.com/python-async</link>
            <pubDate>Mon, 24 Dec 2024 09:00:00 GMT</pubDate>
            <guid>python-async-001</guid>
            <category>Python</category>
            <category>异步编程</category>
        </item>
        
        <item>
            <title>机器学习入门指南</title>
            <description>从零开始学习机器学习的基础概念和实践</description>
            <link>https://techblog.example.com/ml-guide</link>
            <pubDate>Sun, 23 Dec 2024 14:30:00 GMT</pubDate>
            <guid>ml-guide-001</guid>
            <category>机器学习</category>
            <category>人工智能</category>
        </item>
        
        <item>
            <title>Web开发最佳实践</title>
            <description>现代Web开发中的设计模式和最佳实践</description>
            <link>https://techblog.example.com/web-best-practices</link>
            <pubDate>Sat, 22 Dec 2024 11:15:00 GMT</pubDate>
            <guid>web-practices-001</guid>
            <category>Web开发</category>
        </item>
    </channel>
</rss>'''

# 解析RSS
rss_parser = RSSParser()
parsed_rss = rss_parser.parse_rss(sample_rss)

print(f"   RSS解析结果:")
print(f"   频道标题: {parsed_rss.get('title', 'N/A')}")
print(f"   描述: {parsed_rss.get('description', 'N/A')}")
print(f"   文章数量: {len(parsed_rss.get('items', []))}")

print(f"\n   文章列表:")
for item in parsed_rss.get('items', []):
    print(f"     - {item.get('title', 'N/A')} ({', '.join(item.get('categories', []))})")

# 按分类提取文章
python_articles = rss_parser.extract_articles_by_category(parsed_rss, 'Python')
print(f"\n   Python相关文章: {len(python_articles)} 篇")
for article in python_articles:
    print(f"     - {article.get('title', 'N/A')}")

print(f"\n✅ XML基础操作完成")
print(f"🎯 学习目标达成:")
print(f"   ✓ 掌握ElementTree的基本操作和方法")
print(f"   ✓ 熟练进行XML解析和数据提取")
print(f"   ✓ 理解XML的层级结构和命名空间")
print(f"   ✓ 能够创建和修改XML文档")
print(f"   ✓ 掌握实际应用场景中的RSS解析")

## 📋 学习总结

### ✅ 知识清单达成情况验证

**4.4 JSON与XML处理 [⭐基础]**
- ✅ 掌握json模块的基本用法和数据转换
- ✅ 熟练使用xml.etree.ElementTree处理XML数据
- ✅ 理解JSON schema验证和错误处理
- ✅ 能够处理复杂的数据结构和嵌套格式
- ✅ 掌握XML解析、创建和修改的方法
- ✅ 理解XML的层级结构和命名空间处理
- ✅ 能进行数据序列化和反序列化操作
- ✅ 掌握实际应用场景(配置文件、RSS解析等)
- ✅ 能独立处理JSON/XML数据转换任务

### 🎯 与LangChain学习的关联

**数据格式处理重要性**：
- JSON处理是LangChain结构化输出的基础
- XML处理对于LangChain的文档解析和集成至关重要
- 数据序列化影响LangChain的缓存和存储机制
- 格式转换在LangChain的数据管道中发挥重要作用

**实际应用场景**：
- LangChain的输出解析器需要JSON格式处理
- 文档加载器中的XML解析功能
- 配置管理和参数传递使用JSON格式
- API集成中的数据格式转换

### 📚 进阶学习建议

1. **练习建议**：
   - 多练习复杂JSON/XML结构的解析和转换
   - 深入学习数据验证和错误处理技巧
   - 掌握大数据量的流式处理方法

2. **扩展学习**：
   - 学习JSON Schema规范和验证库
   - 了解XPath和XSLT高级XML处理技术
   - 探索YAML、TOML等其他配置格式

3. **实际应用**：
   - 构建配置文件管理系统
   - 开发API数据处理管道
   - 实现文档格式转换工具

### 🔧 常见错误与注意事项

1. **JSON编码问题**：
   ```python
   # 错误：忽略编码问题
   json.dumps({"name": "张三"})  # 输出Unicode转义字符
   
   # 正确：处理中文编码
   json.dumps({"name": "张三"}, ensure_ascii=False)
   ```

2. **XML特殊字符处理**：
   ```python
   # 错误：直接包含特殊字符
   ET.SubElement(root, "content").text = "Hello <world> & everyone"  # 会出错
   
   # 正确：转义特殊字符
   import html
   content = html.escape("Hello <world> & everyone")
   ET.SubElement(root, "content").text = content
   ```

3. **循环引用问题**：
   ```python
   # 错误：循环引用导致序列化失败
   data = {}
   data['self'] = data
   json.dumps(data)  # ValueError: Circular reference detected
   
   # 正确：避免循环引用或使用自定义编码器
   class SafeEncoder(json.JSONEncoder):
       def default(self, obj):
           if id(obj) in self.seen:
               return "[Circular Reference]"
           self.seen.add(id(obj))
           return super().default(obj)
   ```

4. **XML命名空间问题**：
   ```python
   # 错误：忽略命名空间
   root.find('book')  # 无法找到带命名空间的元素
   
   # 正确：使用命名空间前缀
   namespaces = {'ns': 'http://example.com/namespace'}
   root.find('ns:book', namespaces)
   ```

5. **数据类型不一致**：
   ```python
   # 错误：假设JSON数字总是Python int
   data = json.loads('{"price": 99.99}')
   assert isinstance(data['price'], int)  # AssertionError
   
   # 正确：检查数字类型
   price = data['price']
   if isinstance(price, float):
       print(f"价格: {price:.2f}")
   ```

6. **性能问题**：
   ```python
   # 错误：频繁的文件读写
   def save_config(config):
       with open('config.json', 'w') as f:
           json.dump(config, f)
   
   # 正确：批量更新后一次性保存
   def update_config(updates):
       config = load_config()
       config.update(updates)
       with open('config.json', 'w') as f:
           json.dump(config, f)
   ```

---

**🎉 恭喜完成JSON与XML处理学习！**

你已经掌握了Python JSON和XML处理的核心技能，能够熟练进行数据序列化、反序列化、格式转换等操作，掌握了配置文件管理、RSS解析等实际应用技术，为后续学习LangChain的数据处理功能奠定了坚实基础。

## 🚀 下一步学习预告

**继续第四节：常用标准库**：
- 4.5 网络请求处理 [⭐⭐进阶]
- 4.6 多线程与多进程 [⭐⭐进阶]
- 4.7 数据序列化与存储 [⭐⭐进阶]
- 4.8 系统信息获取 [⭐⭐进阶]
- 4.9 常用工具模块 [⭐⭐进阶]

继续加油，完成Python标准库的学习！